[Reference](https://medium.com/@ajosegun_/real-time-dashboard-in-python-b8c9a9c4e050)

In [1]:
# Import necessary modules

import requests
import pandas as pd
import plotly.express as px
import datetime

In [2]:
city_bike_networks = requests.get("http://api.citybik.es/v2/networks").json()

In [3]:
def get_city_data(city):
    
  city_bike_networks = requests.get("http://api.citybik.es/v2/networks").json()

  list_of_dicts = []
  for city_bike_dict in city_bike_networks['networks']:
      new_city = city_bike_dict['location']['city']
      if new_city.lower() == city.lower():
          list_of_dicts.append(city_bike_dict)
          
  return list_of_dicts

In [4]:
city_names = ['Stavanger', 'Copenhagen', 'Utrecht', 'Alhama de Murcia', 'Aranjuez', 'Paris']

for each_city in city_names:
    print(get_city_data(each_city))
    print("-----------------------------\n")

[{'company': ['Gobike A/S'], 'href': '/v2/networks/bysykkelen', 'id': 'bysykkelen', 'location': {'city': 'Stavanger', 'country': 'NO', 'latitude': 58.969975, 'longitude': 5.733107}, 'name': 'Bysykkelen'}]
-----------------------------

[{'company': ['Gobike A/S'], 'href': '/v2/networks/bycyklen', 'id': 'bycyklen', 'location': {'city': 'Copenhagen', 'country': 'DK', 'latitude': 55.673582, 'longitude': 12.564984}, 'name': 'Bycyklen'}]
-----------------------------

[{'company': ['Gobike A/S'], 'href': '/v2/networks/nu-connect', 'id': 'nu-connect', 'location': {'city': 'Utrecht', 'country': 'NL', 'latitude': 52.117, 'longitude': 5.067}, 'name': 'Nu-Connect'}]
-----------------------------

[{'company': ['Domoblue'], 'href': '/v2/networks/onroll-alhama-de-murcia', 'id': 'onroll-alhama-de-murcia', 'location': {'city': 'Alhama de Murcia', 'country': 'ES', 'latitude': 37.849831, 'longitude': -1.424891}, 'name': 'Onroll'}]
-----------------------------

[{'company': ['Domoblue'], 'href': '/v2/

In [5]:
get_city_data("Paris")

[{'company': None,
  'ebikes': True,
  'gbfs_href': 'https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/gbfs.json',
  'href': '/v2/networks/velib',
  'id': 'velib',
  'license': {'name': 'OPEN LICENCE 2.0',
   'url': 'etalab.gouv.fr/wp-content/uploads/2018/11/open-licence.pdf'},
  'location': {'city': 'Paris',
   'country': 'FR',
   'latitude': 48.856614,
   'longitude': 2.3522219},
  'name': "Velib' Métropôle"}]

In [6]:
def get_stations_info(city):
    
    station_dict = get_city_data(city)
    if not station_dict:
        print("Error: No bike company found for {}".format(city))
        return None

    network_address = station_dict[0]['href']
    url = "http://api.citybik.es/{}".format(network_address)
    return requests.get(url).json()['network']['stations']

In [7]:
city = "Paris"
station_info = get_stations_info(city)
station_info

[{'empty_slots': 21,
  'extra': {'banking': False,
   'ebikes': 1,
   'last_updated': 1652397507,
   'renting': 1,
   'returning': 1,
   'station_id': 213688169,
   'uid': '16107'},
  'free_bikes': 14,
  'id': '78ec9186acd18a0b30bd3156d24b9f8d',
  'latitude': 48.865983,
  'longitude': 2.275725,
  'name': 'Benjamin Godard - Victor Hugo',
  'timestamp': '2022-05-13T00:18:03.514000Z'},
 {'empty_slots': 53,
  'extra': {'banking': True,
   'ebikes': 1,
   'last_updated': 1652397672,
   'payment': ['creditcard'],
   'renting': 1,
   'returning': 1,
   'station_id': 99950133,
   'uid': '6015'},
  'free_bikes': 2,
  'id': '43c856353b954711f2bbee185a1f9d04',
  'latitude': 48.85375581057431,
  'longitude': 2.3390958085656166,
  'name': 'André Mazet - Saint-André des Arts',
  'timestamp': '2022-05-13T00:18:03.381000Z'},
 {'empty_slots': 30,
  'extra': {'banking': True,
   'ebikes': 3,
   'last_updated': 1652397574,
   'payment': ['creditcard'],
   'renting': 1,
   'returning': 1,
   'station_id':

In [8]:
def get_available_stations(city = "Paris"):
    '''
    Takes in the city name and returns a pandas dataframe containing information about the city
    Default city name is Paris
    '''
    station_info = get_stations_info(city)
    
    station_list = []
    for info in station_info:

        a_dict = {
            'Station Name': info['name'],
            'empty_slots' : info['empty_slots'],
            'free_bikes' : info['free_bikes'],
            'ebikes' : info['extra']['ebikes'],
            'payment': ', '.join(info['extra']['payment']) if info['extra']['banking'] else "No" ,
            'latitude' : info['latitude'],
            'longitude' : info['longitude'],
            'timestamp' : info['timestamp'],
            'Unique ID': info['extra']['uid'],
        }
        station_list.append(a_dict)
        
    return pd.DataFrame(station_list)

In [9]:
get_available_stations(city = "Paris").sample(5)

,Station Name,empty_slots,free_bikes,ebikes,payment,latitude,longitude,timestamp,Unique ID
1399,Bichat - Grange aux Belles,18,10,3,creditcard,48.873688,2.365015,2022-05-13T00:18:01.761000Z,10202
869,Halles - Bourdonnais,19,1,0,creditcard,48.860449,2.346016,2022-05-13T00:18:01.087000Z,1021
1275,Champeaux - Gallieni,37,9,7,creditcard,48.864459,2.416178,2022-05-13T00:18:01.992000Z,31705
460,Milton - Manuel,14,8,2,No,48.878249,2.341181,2022-05-13T00:18:01.674000Z,9115
350,Ouest - Château,19,4,3,creditcard,48.836173,2.319392,2022-05-13T00:18:01.897000Z,14034


In [10]:
ACCESS_MAP_TOKEN = "pk.eyJ1IjoiYWpvc2VndW4iLCJhIjoiY2t3NnVlZXVrMDIyZjJ1cW1wY2lraGpscSJ9.iPudHyKbx7WXazNrcPH1rA"

In [11]:
def show_map(station_data):
    '''
    Takes in data of the station info in a dataframe format
    
    Shows a map
    '''

    ## Get the current date and time
    current_date = pd.to_datetime(station_data['timestamp'][0]).strftime('%a %d %B, %Y at %H:%M')
        
    map_title = 'Map Showing Number of Bikes in {} at {}'.format(city, current_date)


    # Get access token from ploty
    px.set_mapbox_access_token(ACCESS_MAP_TOKEN)

    fig = px.scatter_mapbox(station_data, lat="latitude", lon="longitude", hover_name="Station Name", color="free_bikes",
                            hover_data=["empty_slots", "free_bikes", "ebikes", "payment"],
                             title=map_title, 
                              color_continuous_scale=px.colors.sequential.Plasma, size_max=20,zoom=12)


    fig.show()

In [12]:
def get_location_data(this_station_data):
    '''
    This method requests input from the biker and returns either of the following 
    - Location data a single location is found for the input
    - No data if the entry is quit  
    
    It displays a list of locations if the input matches more than one location
    - It requests the biker to input the full name of the location from the displayed list
    '''
    
    while True:
        station_name = input("Enter the location you are interested in (quit to exit): ")
        if station_name.strip().lower() == 'quit':
            print("Exiting...")
            return pd.DataFrame(columns = ["Station data is not available"])
#             return None
            
        requested_station_data = this_station_data[this_station_data["name"].map(lambda name: station_name.lower() in name.lower())]

        if len(requested_station_data.index) == 1:
            return requested_station_data
        
        elif requested_station_data.empty:
            print("Error: No data found for the location. Enter a valid location")           
            continue
        else:
            print("Too many values, kindly enter the station full name:")
            for each_data in requested_station_data.index:
                 print(requested_station_data['name'][each_data])
            continue               

In [13]:
live_city_data = get_available_stations("Paris")
show_map(live_city_data)

In [14]:
live_city_data

,Station Name,empty_slots,free_bikes,ebikes,payment,latitude,longitude,timestamp,Unique ID
0,Benjamin Godard - Victor Hugo,21,14,1,No,48.865983,2.275725,2022-05-13T00:18:03.514000Z,16107
1,André Mazet - Saint-André des Arts,53,2,1,creditcard,48.853756,2.339096,2022-05-13T00:18:03.381000Z,6015
2,Faubourg Du Temple - Republique,30,4,3,creditcard,48.867872,2.364898,2022-05-13T00:18:00.418000Z,11037
3,Beaux-Arts - Bonaparte,16,1,1,creditcard,48.856452,2.334852,2022-05-13T00:18:00.755000Z,6021
4,Toudouze - Clauzel,15,6,4,creditcard,48.879296,2.337360,2022-05-13T00:18:00.823000Z,9020
...,...,...,...,...,...,...,...,...,...
1423,Paul Valery - Victor Hugo,31,4,4,No,48.871374,2.288713,2022-05-13T00:18:00.451000Z,16104
1424,Faubourg St-Antoine - Place de la Nation,13,3,3,No,48.848773,2.393606,2022-05-13T00:18:00.882000Z,11201
1425,Quai de Sèvres - Manufacture Nationale,28,0,0,creditcard,48.829135,2.223762,2022-05-13T00:18:00.726000Z,23104
1426,Gare RER les Ardoines,0,48,44,creditcard,48.783856,2.407909,2022-05-13T00:18:00.343000Z,44009


In [15]:
def show_pie_chart(live_data, station_name):

    '''
    Takes in the live data (Pandas dataframe ) and requested station name
    Shows a pie chart of the information
    Returns nothing
    '''
    
    if live_data.empty:
      print("Error: Station data is not available!")
      return None

    requested_station_data = live_data[live_data["Station Name"].map(lambda name: station_name.lower() in name.lower())]
    
    current_date = pd.to_datetime(live_data['timestamp'][0]).strftime('%a %d %B, %Y at %H:%M')

    title = "Bike information for: {0} - Payment type - {1} at {2}".format(
                    requested_station_data["Station Name"].to_string(index=False),
                    requested_station_data["payment"].to_string(index=False),
                    current_date)
    
    data = [int(requested_station_data["empty_slots"]), 
            int(requested_station_data["free_bikes"]), 
            int(requested_station_data["ebikes"]),
             ]

    fig = px.pie(data, values=data, names=["Empty Slots", "Free Bikes", "ebikes"], hole=.3,
                title = title)
    

    fig.update_traces(textposition='inside', textinfo='value+label')

    fig.show()

In [16]:
live_city_data = get_available_stations("Paris")
show_pie_chart(live_city_data, "Gare RER les Ardoines")

In [17]:
def show_bar_chart(live_data, station_name):

    '''
    Takes in the live data (Pandas dataframe ) and requested station name
    Shows a bar chart of the information
    Returns nothing
    '''
    
    if live_data.empty:
        print("Error: Station data is not available!")
        return None

    requested_station_data = live_data[live_data["Station Name"].map(lambda name: station_name.lower() in name.lower())]
    
    current_date = pd.to_datetime(live_data['timestamp'][0]).strftime('%a %d %B, %Y at %H:%M')

    title = "Bike information for: {0} - Payment type - {1} at {2}".format(
                    requested_station_data["Station Name"].to_string(index=False),
                    requested_station_data["payment"].to_string(index=False),
                    current_date)
    
    data = [int(requested_station_data["empty_slots"]), 
            int(requested_station_data["free_bikes"]), 
            int(requested_station_data["ebikes"]),
             ]
    
    fig = px.bar(data,  x=["Empty Slots", "Free Bikes", "ebikes"], y = data, color = data,
                 labels={'y':'Number of Bikes', 'x':""}, title = title                
                 )
    fig.show()


In [18]:
live_city_data = get_available_stations("Paris")
show_bar_chart(live_city_data, "Gare RER les Ardoines")